Chris Bayly:
>That is one weird functional group... a carbamate (common) but connected to an imide! A very highly toxic pesticide, basically a nerve agent, which is my excuse for being less familiar with it. Which is the problematic torsion? In the Google doc text it says "Manual inspection suggests that this transition involves a N-C-O-N torsion, but this needs to be looked at more carefully." The central bond to the N-C-O-N torsion is a single bond, but one which could have somewhat hindered rotation as esters do. Strangely connected to an imide means I do not have a good intuition for the potential barrier. The other candidate hindered rotors are the C-O-N-C (what smirnoff torsion is applied?) and obviously the O-N-C-C (which is a formal double bond).
The key issue to the sampling time must be that higher barriers take longer to sample, so any highly-delocalized acyclic system could be an issue, or sterically-hindered rotors such as 2,6-disubstituted biphenyls. Perhaps one could use some cheminformatics tricks to see when a chemical graph contains hindered rotors of either kind and then do some sampling tricks to address it. Or adaptive sampling without any cheminformatics?


Me:
>All of the following torsion angles were nearly perfectly correlated with the slowest relaxation process:


```
O1 - C2 - N2 - H14
[9, 1, 8, 25]
    periodicity =  2
    phase = 0.0 rad
    force constant =  2.5 kcal/mol

C5 - N2 - C2 - O1
[4, 8, 1, 9]
    periodicity =  2
    phase = 0.0 rad
    force constant =  0.625 kcal/mol

C5 - N2 - C2 - O2
[4, 8, 1, 10]
    periodicity =  2
    phase = 0.0 rad
    force constant =  0.625 kcal/mol

N1 - O2 - C2 - N2
[7, 10, 1, 8]
    periodicity =  2
    phase = 0.0 rad
    force constant =  2.7 kcal/mol

N1 - O2 - C2 - O1
[7, 10, 1, 9]
    periodicity =  2
    phase = 0.0 rad
    force constant =  1.05 kcal/mol```

Chris Bayly:
>Looking at the snapshots says it's all about the carbamate... the "amide" side and the "ester" side of the carbonyl each show transitions. Would this be normative amongst structures containing linear unhindered carbamates: R1-[CX2][NH]C(=O)O[CX2]-R2? Do you have other examples containing this which are well-behaved?

David Mobley:
>the torsion for your carbamate — have you checked if this behaves similarly in GAFF? It would be helpful to know if it’s a problem with the torsion in general or if we’ve botched something in smirnoff99Frosst.

In [6]:
from bayes_implicit_solvent.solvation_free_energy import smiles_list, db, mol_top_sys_pos_list

linear_unhindered_carbamate_smarts = "[CX2][NH]C(=O)O[CX2]"

from openeye import oechem
def look_for_matches(mol, smarts):
    mol_ = oechem.OEMol(mol)

    # Set up query.
    qmol = oechem.OEQMol()
    oechem.OEParseSmarts(qmol, smarts)

    unique = False
    ss = oechem.OESubSearch(qmol)
    matches = []
    for match in ss.Match( mol_, unique):
        # Compile list of atom indices that match the pattern tags
        atom_indices = dict()
        for ma in match.GetAtoms():
            if ma.pattern.GetMapIdx() != 0:
                atom_indices[ma.pattern.GetMapIdx()-1] = ma.target.GetIdx()
        # Compress into list
        atom_indices = [ atom_indices[index] for index in range(len(atom_indices)) ]
        # Store
        matches.append( tuple(atom_indices) )
    return matches


all_linear_unhindered_carbamate_matches = []
for i in range(len(mol_top_sys_pos_list)):
    mol, _, _, _ = mol_top_sys_pos_list[i]
    
    all_linear_unhindered_carbamate_matches.append(look_for_matches(mol, linear_unhindered_carbamate_smarts))

In [8]:
sum([len(m) > 0 for m in all_linear_unhindered_carbamate_matches])

0

In [10]:
from bayes_implicit_solvent.solvation_free_energy import db, mol_top_sys_pos_list, smiles_list
mol, top, sys, pos = mol_top_sys_pos_list[0]

In [11]:
mol.IsValid()

True

In [15]:
[c for c in mol.GetConfs()]

[<oechem.OEConfBase; proxy of <Swig Object of type 'OEChem::OEConfBase *' at 0x11563d300> >]

In [17]:
len(pos)

27

[ ] TODO: Set up aldicarb simulation using GAFF

In [9]:
fname = '../../../MSKCC/Chodera Lab/feedstock/FreeSolv-0.51/mol2files_gaff/mobley_5200358.mol2'

In [59]:
from openeye import oeomega  # Omega toolkit

omega = oeomega.OEOmega()

from openeye import oechem  # OpenEye Python toolkits

from openeye import oequacpac

In [115]:
ifs = oechem.oemolistream()

if ifs.open(fname):
    for mol_ in ifs.GetOEGraphMols():
        mol = oechem.OEMol(mol_)

In [116]:
atoms = [a for a in mol.GetAtomIter()]

In [117]:
a = atoms[0]

In [118]:
[a.GetPartialCharge() for a in atoms]

[-0.09309999644756317,
 -0.024399999529123306,
 -0.09309999644756317,
 0.3833000063896179,
 -0.36340001225471497,
 -0.25440001487731934,
 0.7635999917984009,
 -0.5260999798774719,
 -0.5651999711990356,
 0.09290000051259995,
 -0.311599999666214,
 -0.029500000178813934,
 0.04969999939203262,
 0.04969999939203262,
 0.04969999939203262,
 0.04969999939203262,
 0.04969999939203262,
 0.04969999939203262,
 0.05810000002384186,
 0.3285999894142151,
 0.0478999987244606,
 0.0478999987244606,
 0.0478999987244606,
 0.06419999897480011,
 0.06419999897480011,
 0.06419999897480011]

In [119]:
a.GetSymmetryClass()

0

In [120]:
len(atoms)

26

In [121]:
mol

<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x11620c1e0> >

In [108]:
import numpy as np

In [109]:
coord_dict = mol.GetCoords()
positions = np.array([coord_dict[key] for key in coord_dict])

topology = generateTopologyFromOEMol(mol)
residue = list(topology.residues())[0]

In [110]:
len(positions)

26

In [111]:

residue

<Residue 0 ([(Z)-(2-methyl-2-methylsulfanyl-propylidene)amino]) of chain 0>

In [112]:
len(list(residue.atoms()))

26

In [140]:
from simtk import openmm as mm
from simtk.openmm import app
from openmoltools import forcefield_generators

forcefield = app.ForceField()
forcefield.registerTemplateGenerator(forcefield_generators.gaffTemplateGenerator)

In [148]:
system_generator = forcefield_generators.SystemGenerator([])
system_generator.createSystem(topology)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcf in position 172: invalid continuation byte

In [141]:
mol,_,_,_ = mol_top_sys_pos_list[0]

In [142]:
template, additional_parameters_ffxml = forcefield_generators.generateResidueTemplate(mol, gaff_version='gaff2')
forcefield.registerResidueTemplate(template)
template

In [144]:
system = forcefield.createSystem(topology)

RuntimeError: omega returned error code 0

In [44]:
import parmed

In [46]:
pmol = parmed.load_file(fname)

In [47]:
type(pmol)

parmed.modeller.residue.ResidueTemplate

In [48]:
pmol.coordinates

array([[ 7.670e-01,  3.740e-01,  6.880e-01],
       [ 2.134e+00,  4.190e-01,  4.000e-03],
       [ 2.106e+00,  1.369e+00, -1.195e+00],
       [ 2.604e+00, -9.450e-01, -4.690e-01],
       [ 1.936e+00, -1.647e+00, -1.325e+00],
       [ 7.850e-01, -9.340e-01, -1.688e+00],
       [ 7.400e-02, -1.673e+00, -2.594e+00],
       [ 4.010e-01, -2.779e+00, -3.013e+00],
       [-1.062e+00, -9.810e-01, -2.961e+00],
       [-2.033e+00, -1.480e+00, -3.908e+00],
       [ 3.386e+00,  1.005e+00,  1.206e+00],
       [ 2.653e+00,  2.616e+00,  1.559e+00],
       [ 7.860e-01, -3.060e-01,  1.548e+00],
       [ 4.650e-01,  1.358e+00,  1.068e+00],
       [-1.600e-02,  1.800e-02,  9.000e-03],
       [ 3.076e+00,  1.418e+00, -1.703e+00],
       [ 1.821e+00,  2.387e+00, -9.020e-01],
       [ 1.378e+00,  1.027e+00, -1.940e+00],
       [ 3.551e+00, -1.324e+00, -4.000e-02],
       [-1.224e+00, -7.000e-02, -2.543e+00],
       [-2.799e+00, -2.051e+00, -3.383e+00],
       [-1.542e+00, -2.125e+00, -4.637e+00],
       [-2

In [49]:
pmol.to_structure()

<Structure 26 atoms; 1 residues; 25 bonds; NOT parametrized>

In [159]:
from openmoltools import utils
from simtk.openmm.app import ForceField
molecules = [ mol ]
from openmoltools.forcefield_generators import generateForceFieldFromMolecules
ffxml = generateForceFieldFromMolecules(molecules, gaff_version='gaff2')

gaff_xml_filename = utils.get_data_filename("parameters/gaff2.xml")
forcefield = ForceField(gaff_xml_filename)

# Parameterize the molecules.
from openmoltools.forcefield_generators import generateTopologyFromOEMol
for molecule in molecules:
    # Create topology from molecule.
    topology = generateTopologyFromOEMol(molecule)
    # Create system with forcefield.
    system = forcefield.createSystem(topology)

ValueError: No template found for residue 1 ().  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.

In [157]:
bond_gen = forcefield.getGenerators()[0]
bond_gen.types1

[['ow'],
 ['hw'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['br'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c1'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 ['c2'],
 

In [158]:
bond_gen.k

[512941.664,
 12694.256,
 67789.168,
 189961.968,
 141343.888,
 122574.464,
 112742.064,
 135687.12,
 144373.104,
 129394.384,
 118415.568,
 74031.696,
 114248.304,
 92391.088,
 98985.072,
 81127.76,
 94365.936,
 63303.92,
 161987.744,
 134616.016,
 127093.184,
 111394.816,
 106106.24,
 122072.384,
 115093.472,
 141586.56,
 107779.84,
 143569.776,
 145251.744,
 147293.536,
 700635.904,
 448399.28,
 247575.648,
 272478.816,
 434039.792,
 650042.976,
 640511.824,
 218814.832,
 268236.24,
 403496.592,
 362936.896,
 375062.128,
 128457.168,
 746040.672,
 616512.4,
 335548.432,
 259115.12,
 358870.048,
 317582.336,
 602922.768,
 602922.768,
 342284.672,
 270688.064,
 665239.264,
 353305.328,
 364325.984,
 180732.064,
 170590.048,
 170590.048,
 189920.128,
 318410.768,
 310126.448,
 200020.304,
 214764.72,
 243835.152,
 236647.04,
 403195.344,
 213852.608,
 333356.016,
 366501.664,
 366501.664,
 385354.768,
 385354.768,
 161251.36,
 418441.84,
 232940.016,
 214145.488,
 308009.344,
 329891.6